In [ ]:
import pandas as pd
import networkx as nx
import torch
import random
from pathlib import Path
import numpy as np
from typing import Any

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

Using device: cpu



In [ ]:
dataset = "/Users/vanshgupta/Desktop/AI and ML reading material/GraphGAN_Project/GraphGAN/bio-grid-human/bio-grid-human_dataset.txt1" # used to generate graph and get the number of nodes
filename = "/Users/vanshgupta/Desktop/AI and ML reading material/GraphGAN_Project/Emb and Data/Emb/biogrid-human/DeepWalk/emb.txt" # used to read the file and get embeddings.
error_emb_filename = "/Users/vanshgupta/Desktop/AI and ML reading material/GraphGAN_Project/Emb and Data/Emb/biogrid-human/DeepWalk/emb_10%_error.txt" # path where you want to save the embeddings with induced errors.
embedding_filename = "/Users/vanshgupta/Desktop/AI and ML reading material/GraphGAN_Project/Emb and Data/Emb/biogrid-human/DeepWalk/fuzzy_emb_10%_error.txt" # filepath where you want to store the fuzzy layer processed embeddings

k = 5 # k -> Number of clusters
node_embed_dim = 50 # Dimension of node feature vector

rng = np.random.default_rng(seed = 42) # set seed

In [ ]:
def add_noise(x : torch.Tensor, noise_ratio : float, node_embed_dim : int) -> torch.Tensor:
    """
    Adds Gaussian noise to a subset of nodes in a given tensor.

    This function introduces noise to a fraction of the nodes in the input tensor `x`. The noise is
    generated from a normal distribution with mean 0 and standard deviation 1. The number of nodes
    to be noised is determined by the `noise_ratio` parameter.

    Parameters:
        x (torch.Tensor): The input tensor of shape `(num_nodes, node_embed_dim)` representing
                          node embeddings.
        noise_ratio (float): The fraction of nodes to which noise will be added. Should be a
                             value between 0 and 1.
        node_embed_dim (int): The dimensionality of the node embeddings.

    Returns:
        torch.Tensor: The modified tensor `x` with noise added to a subset of nodes.

    """


    num_nodes = x.shape[0] #fetch the number of nodes
    num_nodes_to_add_noise = int(num_nodes * noise_ratio) #fetch the number of nodes that are to be noised
    total_nodes = torch.arange(0 , num_nodes).tolist() #gives a list of nodes
    nodes_to_noise = random.sample(total_nodes, num_nodes_to_add_noise) #gives the node-ids where noise is to be added.

    noise_tensor = torch.normal(mean = 0,
                                std = 1,
                                size = (num_nodes_to_add_noise, node_embed_dim))

    x[nodes_to_noise] = noise_tensor
    return x


def read_embeddings(filename: Path, n_node: int, n_embed: int) -> np.ndarray:
    """
    Reads pretrained node embeddings from a file and returns them as a numpy array.

    This function initializes an embedding matrix with random values and updates it with
    the pretrained embeddings provided in the file. If a node does not have a pretrained
    embedding in the file, its embedding will remain as the initialized random values.

    Parameters:
        filename (Path): Path to the file containing pretrained node embeddings.
                         The file format is expected to have node embeddings on each
                         line in the format: `node_id dim_1 dim_2 ... dim_n`, with the
                         first line typically skipped (header).
        n_node (int): The total number of nodes in the graph.
        n_embed (int): The embedding dimensionality for each node.

    Returns:
        np.ndarray: A numpy array of shape `(n_node, n_embed)` representing the node embeddings.

    """

    # Open the file and read all lines, skipping the first line (header)
    with open(filename, "r") as f:
        lines = f.readlines()[1:]

        # Create an embedding matrix initialized with random normal values
        embedding_matrix = rng.standard_normal(size=(n_node, n_embed))

        # Iterate over each line in the file (representing a node embedding)
        for line in lines:
            emd = line.split()  # Split the line into node ID and embedding values
            # Update the embedding matrix for the specific node ID
            embedding_matrix[int(emd[0]), :] = str_list_to_float(emd[1:])

        # Return the final embedding matrix
        return embedding_matrix

def save_embeddings(filepath : Path, graph : Any, embeddings : np.ndarray) -> None:
    """
    Saves node embeddings to a file in a specified format.

    This function takes a graph, its corresponding embeddings, and a file path. It maps the
    embeddings to the graph nodes, combines the node indices with their respective embeddings,
    and writes the data to the file. The output file includes a header indicating the total
    number of nodes and the embedding dimensionality, followed by the embeddings for each node.

    Parameters:
        filepath (Path): Path to the file where embeddings will be saved.
        graph (Any): A graph object (e.g., from NetworkX) containing node information.
        embeddings (np.ndarray): A 2D numpy array of shape `(num_nodes, embedding_dim)`
                                  containing node embeddings.

    File Format:
        The saved file has the following format:
        - The first line contains the number of nodes and the embedding dimensionality,
          separated by a tab.
        - Each subsequent line corresponds to a node, containing the node ID followed
          by its embedding values separated by spaces.
    """
    # Map the embeddings to the node indices in the graph
    new = embeddings[torch.tensor(np.array(graph.nodes()))]

    # Get the node indices and reshape them into a column vector
    index = np.array(graph.nodes()).reshape(-1, 1)

    # Combine node indices with their embeddings into a single matrix
    embedding_matrix = np.hstack([index, new])

    # Convert the embedding matrix into a list of strings for saving
    embedding_list = embedding_matrix.tolist()
    embedding_str = [str(int(emb[0])) + " " + " ".join([str(x) for x in emb[1:]]) + "\n"
                     for emb in embedding_list]

    # Write the embeddings to the file
    with open(filepath, "w+") as f:
        # Include the header with number of nodes and embedding dimensionality
        lines = [str(graph.number_of_nodes()) + "\t" + str(embeddings.shape[1]) + "\n"] + embedding_str
        f.writelines(lines)


def str_list_to_float(str_list : list[str]) -> list[float]:
    """Convert the string items of a list to float items"""
    return [float(item) for item in str_list]

In [ ]:

df = pd.read_csv(dataset,
                sep = '\t',
                names = ["NodeIDfrom", "NodeIDto"])
#create the graph networkx object from the above dataframe

G = nx.from_pandas_edgelist(df = df,
                             source = "NodeIDfrom",
                             target = "NodeIDto",
                             create_using=nx.Graph())

X = read_embeddings(filename=filename,
                                     n_node = len(G),
                                     n_embed = 50)

print(f"Number of Nodes in the graph : {len(G)}")
print(f"The shape of the embedding vector {X.shape}")

9436

In [ ]:
### ------------------------------------------------------------ TO ADD NOISE ------------------------------------------------------------ ###
# Run this when you have to add noise to the original embeddings.
X = add_noise(x = X, noise_ratio=0.1, node_embed_dim=50)
X = np.array(X)
save_embeddings(filepath=error_emb_filename, graph=G, embeddings=X)


In [ ]:
class KMeansNodeClustering:
    """
    Implements K-Means clustering for node embedding vectors.

    This class clusters a given set of node embedding vectors into `k` clusters using the
    K-Means algorithm. Each node embedding is assigned to the cluster whose centroid
    is nearest to it, based on the Euclidean distance.

    Attributes:
        X (np.ndarray): Dataset consisting of node embeddings of shape (num_nodes, node_embed_dim).
        k (int): Number of clusters to form.
        node_embed_dim (int): Dimensionality of each node embedding.
        centroid (np.ndarray): Centroids of the clusters, updated during training.

    Methods:
        euclidean_measure(centroid, node_embed):
            Static method to compute the Euclidean distance between a centroid and a node embedding.

        dimensional_mean(cluster_num, node_embed_dim, cluster_indices, cluster_centres):
            Computes the mean embedding vector for a given cluster and updates the cluster center.

        fit(max_iterations=200):
            Fits the K-Means model to the node embedding vector, returning cluster assignments and indices.
    """
    def __init__(self, X, k, node_embed_dim):
        """
        Initializes the KMeansNodeClustering object.

        Parameters:
            X (np.ndarray): The input dataset containing node embeddings.
            k (int): The number of clusters to create.
            node_embed_dim (int): Dimensionality of the node embeddings.
        """
        self.k = k  # Number of clusters
        self.X = X  # Dataset of node embeddings
        self.node_embed_dim = node_embed_dim  # Embedding dimensionality
        self.centroid = None  # Placeholder for cluster centroids

    @staticmethod
    def euclidean_measure(centroid, node_embed):
        """
        Calculates the Euclidean distance between a centroid and a node embedding.

        Parameters:
            centroid (np.ndarray): Centroid of a cluster (shape: (k, node_embed_dim)).
            node_embed (np.ndarray): Node embedding (shape: (node_embed_dim,)).

        Returns:
            np.ndarray: Euclidean distances from the node embedding to each centroid.
        """
        return np.sqrt(np.sum((centroid - node_embed) ** 2, axis=1))

    @staticmethod
    def dimensional_mean(cluster_num, node_embed_dim, cluster_indices, cluster_centres):
        """
        Computes the mean embedding for a given cluster, dimension-wise.

        Parameters:
            cluster_num (int): The cluster number for which the mean is calculated.
            node_embed_dim (int): Dimensionality of the embeddings.
            cluster_indices (list): Indices of nodes belonging to each cluster.
            cluster_centres (list): List to store the new cluster centroids.

        Returns:
            None: Updates the cluster_centres list with the mean centroid.
        """
        axis_centre = np.zeros(node_embed_dim)  # Initialize centroid
        for i in range(node_embed_dim):  # Iterate over dimensions
            y = 0
            tup = cluster_indices[cluster_num].shape
            shape = tup[0]  # Number of nodes in the cluster
            for x in range(shape):  # Sum values for each dimension
                y += X[cluster_indices[cluster_num][x]][0][i]
            mean = np.mean(y)  # Calculate mean for the dimension
            axis_centre[i] += mean
        axis_centre = axis_centre / np.linalg.norm(axis_centre)  # Normalize centroid
        return cluster_centres.append(axis_centre)

    def fit(self, max_iterations=200):
        """
        Fits the K-Means model to the dataset, clustering nodes into `k` clusters.

        Parameters:
            max_iterations (int): Maximum number of iterations for the K-Means algorithm.

        Returns:
            y (np.ndarray): Array indicating the cluster assignment for each node.
            cluster_indices (np.ndarray): Array of lists, where each list contains
                                           the indices of nodes in the respective cluster.
        """
        # Initialize centroids randomly
        self.centroid = rng.standard_normal(size=(self.k, self.node_embed_dim))
        self.centroid = self.centroid / (np.max(self.centroid))  # Normalize centroids

        # K-Means iterations
        for _ in range(max_iterations):
            y = []  # To store cluster assignments

            # Assign each node to the nearest cluster
            for node_embed in self.X:
                distance = KMeansNodeClustering.euclidean_measure(
                    node_embed=np.array(node_embed),
                    centroid=self.centroid
                )
                cluster_num = np.argmin(distance)  # Closest centroid
                y.append(cluster_num)

            y = np.array(y)  # Convert assignments to a numpy array

            # Create lists to store node indices for each cluster
            cluster_indices = []
            for i in range(self.k):
                cluster_indices.append(np.argwhere(y == i))
            cluster_indices = np.array(cluster_indices, dtype=object)

            # Update cluster centroids
            cluster_centres = []
            for j, indices in enumerate(cluster_indices):
                if len(indices) == 0:  # If cluster is empty, retain old centroid
                    cluster_centres.append(self.centroid[j])
                else:
                    KMeansNodeClustering.dimensional_mean(
                        cluster_num=j,
                        node_embed_dim=self.node_embed_dim,
                        cluster_indices=cluster_indices,
                        cluster_centres=cluster_centres
                    )

            # Update centroids for the next iteration
            self.centroid = np.array(cluster_centres)

        return y, cluster_indices



In [ ]:
clustered_nodes = KMeansNodeClustering(X = X,
                                       k = k,
                                       node_embed_dim = node_embed_dim)
#cluster_indices_id stores the number_id of the clusters each embeddings is related to
#cluster_indices stores all the nodes that belong to one particular cluster, for all the clusters.
cluster_indices_id, cluster_indices = clustered_nodes.fit()


In [ ]:
centroid_array = clustered_nodes.centroid #contains the centroids of all the clusteres.
centroid_array

In [ ]:
cluster_indices = np.array(cluster_indices, dtype=object)
cluster_indices_array = np.array([torch.tensor(np.squeeze(cluster_indices[0], axis = 1)),
                                  torch.tensor(np.squeeze(cluster_indices[1], axis = 1)),
                                  torch.tensor(np.squeeze(cluster_indices[2], axis = 1)),
                                  torch.tensor(np.squeeze(cluster_indices[3], axis = 1)),
                                  torch.tensor(np.squeeze(cluster_indices[4], axis = 1)),
                                 ], dtype = object)

cluster_indices_array

In [ ]:
class Antecedant:
    """
    Implements the antecedent part of the fuzzy logic system. This component computes membership
    functions for the node embeddings and handles operations like calculating dot products, means,
    variances, and Gaussian Membership Functions (GaussianMF).

    Attributes:
        X (torch.Tensor): Dataset of node embeddings. Inherited from FuzzyLayer.
        centroid_array (torch.Tensor): Array of cluster centroids. Inherited from FuzzyLayer.
        cluster_indices_array (list): List of cluster indices. Inherited from FuzzyLayer.

    Methods:
        dot_product():
            Computes the dot product of the dataset `X` with the transposed centroid array.

        get_points(cluster_num, embed_axis):
            Fetches all points from the dataset corresponding to a given cluster and embedding axis.

        get_mean(cluster_num, embed_axis):
            Retrieves the mean value (centroid) for a specific cluster and embedding axis.

        get_standard_deviation(cluster_num, embed_axis):
            Calculates the standard deviation of points in a specific cluster along a specified axis.

        get_stddev_tensor(embed_axis):
            Computes the standard deviations for all clusters along a specified embedding axis.

        gaussianMF(cluster_num, embed_axis, element):
            Computes the Gaussian Membership Function (GMF) for a node embedding, given the cluster,
            axis, and element value.

        get_membership_array(embed_axis):
            Generates a membership array for all node embeddings along a specified embedding axis.
            Each element represents the membership degree of a node to a cluster.

    """
    def __init__(self, X, centroid_array, cluster_indices_array):
              """
        Initializes the Antecedant object.

        Parameters:
            X (np.ndarray): Dataset of node embeddings. Shape: (num_nodes, node_embed_dim).
            centroid_array (np.ndarray): Centroid array with cluster centroids. Shape: (num_clusters, node_embed_dim).
            cluster_indices_array (list): A list of lists, where each sublist contains indices of nodes
                                           belonging to a specific cluster.
        """
        self.X = torch.tensor(X, device=device)
        self.centroid_array = torch.tensor(centroid_array, device=device)
        self.cluster_indices_array = cluster_indices_array

    def dot_product(self):
        """
        Computes the dot product of the dataset (X) with the transposed centroid array.

        Returns:
            torch.Tensor: A matrix where each element represents the dot product of a node embedding
                        with a centroid. Shape: (num_nodes, num_clusters).
        """
        # Perform matrix multiplication for efficiency
        dot_products = torch.matmul(self.X, self.centroid_array.T)
        return dot_products

    def get_points(self, cluster_num, embed_axis):
        """
        Fetches all points from the dataset corresponding to a specific cluster and embedding axis.

        Args:
            cluster_num (int): The index of the cluster whose points need to be retrieved.
            embed_axis (int): The specific embedding axis to extract the points from.

        Returns:
            torch.Tensor: A tensor containing the points of the specified cluster along the given axis.
        """
        # Retrieve node indices belonging to the specified cluster
        cluster_ids = torch.tensor(self.cluster_indices_array[cluster_num], device=device)
        # Fetch points for the given axis
        points = self.X[cluster_ids, embed_axis]
        return points

    def get_mean(self, cluster_num, embed_axis):
        """
        Retrieves the mean value (centroid) for a specific cluster and embedding axis.

        Args:
            cluster_num (int): The cluster index.
            embed_axis (int): The specific embedding axis.

        Returns:
            torch.Tensor: The mean value for the specified cluster and axis.
        """
        # Access the mean directly from the centroid array
        return self.centroid_array[cluster_num, embed_axis]

    def get_standard_deviation(self, cluster_num, embed_axis):
        """
        Computes the standard deviation of points in a specific cluster along a specified axis.

        Args:
            cluster_num (int): The cluster index.
            embed_axis (int): The specific embedding axis.

        Returns:
            torch.Tensor: The standard deviation of points along the given axis for the specified cluster.
        """
        # Retrieve points for the specified cluster and axis
        points = self.get_points(cluster_num, embed_axis)
        # Compute the mean
        mean = self.get_mean(cluster_num, embed_axis)
        # Calculate variance and its square root (standard deviation)
        variance = torch.var(points)
        variance = torch.sqrt(variance)
        return variance

    def get_stddev_tensor(self, embed_axis):
        """
        Computes the standard deviations for all clusters along a specified embedding axis.

        Args:
            embed_axis (int): The embedding axis for which standard deviations are computed.

        Returns:
            torch.Tensor: A tensor containing the standard deviations for all clusters along the given axis.
                        Shape: (num_clusters,).
        """
        # Compute standard deviation for all clusters in a vectorized manner
        stddev = torch.tensor([self.get_standard_deviation(cluster_num, embed_axis)
                            for cluster_num in range(self.centroid_array.shape[0])], device=device)
        return stddev

    def gaussianMF(self, cluster_num, embed_axis, element):
        """
        Computes the Gaussian Membership Function (GMF) for a node embedding.

        Args:
            cluster_num (int): The cluster index.
            embed_axis (int): The specific embedding axis.
            element (float): The value of the embedding element to compute membership for.

        Returns:
            torch.Tensor: The membership value of the element to the specified cluster using GMF.
        """
        # Retrieve the mean and standard deviation
        mean = self.get_mean(cluster_num, embed_axis)
        stddev_tensor = self.get_stddev_tensor(embed_axis)
        # Handle zero standard deviation cases
        if stddev_tensor[cluster_num] != 0:
            gaussian = (element - mean) ** 2 / (2 * (stddev_tensor[cluster_num] ** 2))
        else:
            return 0.0
        # Compute the exponential Gaussian function
        return torch.exp(-gaussian)

    def get_membership_array(self, embed_axis):
        """
        Generates a membership array for all node embeddings along a specified embedding axis.

        Args:
            embed_axis (int): The embedding axis for which membership values are computed.

        Returns:
            torch.Tensor: A 2D tensor where each row corresponds to a node and each column represents
                        the membership degree of the node to a specific cluster.
                        Shape: (num_nodes, num_clusters).
        """
        # Preallocate the membership array with zeros
        membership_array = torch.zeros((self.X.shape[0], self.centroid_array.shape[0]), device=device)
        # Iterate over all nodes
        for node_embed_num in range(self.X.shape[0]):
            # Iterate over all clusters
            for i in range(self.centroid_array.shape[0]):
                # Compute membership using the Gaussian Membership Function
                membership = self.gaussianMF(cluster_num=i, embed_axis=embed_axis,
                                            element=self.X[node_embed_num, embed_axis])
                membership_array[node_embed_num, i] = membership
        return membership_array


In [ ]:
b = Antecedant(X = X,
               centroid_array = centroid_array,
               cluster_indices_array = cluster_indices_array)


In [ ]:
# calcualtes the 3-D membership array.
membership_array = []
for embed_axis in range(node_embed_dim):
    mem_array_per_axis = b.get_membership_array(embed_axis = embed_axis)
    membership_array.append(mem_array_per_axis)
membership_array = torch.stack(membership_array)
membership_array

/var/folders/3r/s7htwqw902q6tm435h5z7f2c0000gn/T/ipykernel_91822/1769519963.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cluster_ids = torch.tensor(self.cluster_indices_array[cluster_num], device=device)


tensor([[[0.8075, 0.9054, 0.9412, 0.9210, 0.9588],
         [0.5085, 0.7430, 0.8708, 0.7947, 0.9348],
         [0.9568, 0.9788, 0.9845, 0.9816, 0.9871],
         ...,
         [0.7661, 0.8840, 0.9286, 0.9033, 0.9508],
         [0.6774, 0.8358, 0.9003, 0.8634, 0.9328],
         [0.0092, 0.1240, 0.3545, 0.1940, 0.5735]],

        [[0.9992, 0.9905, 0.9523, 0.9995, 0.9881],
         [0.8668, 0.9805, 0.9984, 0.9797, 1.0000],
         [0.6290, 0.8867, 0.9866, 0.9209, 0.9895],
         ...,
         [0.7951, 0.9573, 0.9999, 0.9642, 0.9986],
         [0.9348, 0.9965, 0.9897, 0.9925, 0.9988],
         [0.9525, 0.9989, 0.9852, 0.9953, 0.9978]],

        [[0.9734, 0.9114, 1.0000, 0.9637, 1.0000],
         [0.9385, 0.9995, 0.9619, 0.9999, 0.9688],
         [0.9909, 0.9355, 0.9992, 0.9749, 0.9988],
         ...,
         [0.9989, 0.9544, 0.9966, 0.9834, 0.9964],
         [1.0000, 0.9640, 0.9942, 0.9875, 0.9944],
         [0.1684, 0.2995, 0.7196, 0.5667, 0.7960]],

        ...,

        [[0.6354, 0.

In [ ]:
### --------------------------------------------------- OPTIONAL ---------------------------------------------------###
# Run this code to eliminate any Nan values in the membership array.
# Chose the dimensions accordingly.

#membership_array = membership_array[:, :, [0, 1, 2, 3]]
#membership_array

In [ ]:
class Consequent():
    """
    This class generates the crisp embeddings using the membership values generated by the Antecedant class.

    Essentially, we fuzzified each of the embeddings for each node, clustered the nodes, found centroids,
    and calculated membership for each embedding. Now, we use that membership to output crisp memberships.

    To do that, we generate sets with variances and treat them as the consequent fuzzy set from which crisp output is generated.
    """

    def __init__(self, X, centroid_array, cluster_indices_array):
        """
        Parameters:
            X (np.ndarray): Dataset of node embeddings. Shape: (num_nodes, node_embed_dim).
            centroid_array (np.ndarray): Centroid array with cluster centroids. Shape: (num_clusters, node_embed_dim).
            cluster_indices_array (list): A list of lists, where each sublist contains indices of nodes
                                           belonging to a specific cluster.
        """

        self.X = torch.tensor(X, device=device)
        self.centroid_array = torch.tensor(centroid_array, device=device)
        self.cluster_indices_array = cluster_indices_array

    def get_average_membership_array(self, membership_array):
        """
        Calculates the average membership for each embedding dimension across all clusters.

        Args:
            membership_array (torch.Tensor): The membership values of shape (node_embed_dim, num_nodes, num_clusters).

        Returns:
            torch.Tensor: The average membership values of shape (node_embed_dim, num_nodes).
        """
        return torch.mean(membership_array, dim=2)  # shape = (node_embed_dim, num_nodes)

    def get_embed_axis_mean(self):
        """
        Calculates the mean of each embedding axis.

        Returns:
            torch.Tensor: The mean values for each embedding axis of shape (node_embed_dim,).
        """
        a = self.X.transpose(dim0=1, dim1=0) #shape = (50, 5242)
        return torch.mean(a, axis = 1) #shape = (50,)

    def get_embed_axis_variance(self):
        """
        Calculates the variance of each embedding axis.

        Returns:
            torch.Tensor: The variance values for each embedding axis of shape (node_embed_dim,).
        """
        a = self.X.transpose(dim0=1, dim1=0)
        mean = self.get_embed_axis_mean()
        variance = torch.zeros(size = (node_embed_dim,))
        for i in range(node_embed_dim):
            x = torch.sqrt(torch.mean((a[i] - mean[i])**2))
            variance[i] += x
        return variance #shape = (50,)

    def get_crisp_embeddings(self, average_membership_array):
        """
        Generates the crisp embeddings using the average membership values.

        Args:
            average_membership_array (torch.Tensor): The average membership values of shape (node_embed_dim, num_nodes).

        Returns:
            torch.Tensor: The crisp embeddings of shape (num_nodes, node_embed_dim).
        """
        a = self.X.transpose(dim0=1, dim1=0)
        mean = self.get_embed_axis_mean()
        variance = self.get_embed_axis_variance()
        new_X = torch.zeros(size = (node_embed_dim, self.X.shape[0]))
        for i in range(node_embed_dim):
            for j in range(self.X.shape[0]):
                if average_membership_array[i][j] == 0:
                    new_X[i][j] = a[i][j]
                else:
                    b = -1*torch.log(average_membership_array[i][j])
                    if(a[i][j] < 0):
                        b = -1*torch.sqrt(b)
                    else:
                        b = torch.sqrt(b)
                    b = b*variance[i] + mean[i]
                    new_X[i][j] = b
        new_X = torch.transpose(new_X, dim0=1, dim1=0)
        new_X.shape
        #new_X = new_X/np.linalg.norm(new_X, axis = 0)
        return new_X

In [ ]:
c = Consequent(X = X,
               centroid_array=centroid_array,
               cluster_indices_array = cluster_indices_array)

In [ ]:
average_membership = c.get_average_membership_array(membership_array = membership_array)
crisp = c.get_crisp_embeddings(average_membership_array = average_membership)
save_embeddings(filepath = embedding_filename, graph = G, embeddings = crisp)